# Pseudo code for performing Bayesian Optimization Simulation

## 1. Importing packages and dependencies

Usefull packages for handling the data and plotting the results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

Does Python have the access to the GPU? If true, faster calculations are performed

In [ ]:
SMOKE_TEST = os.environ.get("SMOKE_TEST")
print(f'SMOKE_TEST: {SMOKE_TEST}')

Importing Bofire dependencies:

## 2. Setting up the problem using Bofire

In [ ]:
from bofire.data_models.domain.api import Domain
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput

Consider the problem where we need to maximize a target variable over two different variables here specified as features:

In [ ]:
lower_bound1 = 30
upper_bound1 = 55
feature1 = ContinuousInput(key='feature1', bounds = [lower_bound1, upper_bound1], unit='°C')

lower_bound2 = 7
upper_bound2 = 9 
feature2 = ContinuousInput(key='feature2', bounds=[lower_bound2, upper_bound2])

input_features = Inputs(
    features=[
        feature1,
        feature2
    ]
)

Using the module MaximizeObjective it is possible to obtain the maximum out of the target.

In [ ]:
from bofire.data_models.objectives.api import MaximizeObjective
# from bofire.data_models.objectives.api import MinimizeObjective

target_lower = 0
target_upper = 100
objective = MaximizeObjective(
    w=1.0, # weight of the objective: in the context of a Multiple-Objective Optimization, we can specify the weight of each objective
    lower_bound=target_lower,
    upper_bound=target_lower
)
target_feature = ContinuousOutput(key="Target", objective=objective, unit='%')

# create an output feature
output_features = Outputs(features=[target_feature])

Set up the domain of the problem of interest:

In [ ]:
domain = Domain(
    inputs = input_features,
    outputs = output_features,
)
domain.get_feature_reps_df() # returns the entire set of features and target with boundaries

In [ ]:
from bofire.data_models.strategies.api import SoboStrategy # Single-Objective Bayesian Optimization
from bofire.data_models.acquisition_functions.api import qEI # quasi-MonteCarlo-Expected of Improvement Acquisition Function
import bofire.strategies.mapper as StrategyMapper # map the data to the Bayesian Optimization framework

In [ ]:
qExpectedImprovement = qEI() # instance of qEI

# puttin all the information together
sobo_strategy_data_model = SoboStrategy(
    domain=domain,
    acquisition_function=qExpectedImprovement,
)

# map the strategy data model to the actual strategy that has functionality
sobo_strategy = StrategyMapper.map(sobo_strategy_data_model)

## 3. Setting up the simulation

Importing the packages and modules for fitting a second order model:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)

Importing the actual dataset:

In [ ]:
path  = r"C:\path\to\complete_df.csv"
df = pd.read_csv(path)

X = df[['Feature1', 'Feature2']] # predictors 
y = df['Target'] # target variable

Fitting the second order regression model which will be used for simulating experiment results:

In [ ]:
X_poly = poly_features.fit_transform(X)
reg = LinearRegression()
reg.fit(X_poly, y)

Starting dataset wich will be used as well in the other simulation for comparison reasons.

In [ ]:
df_initial = pd.DataFrame({'Feature1': [35, 35, 40, 40, 37.5], 'Feature2': [7, 7.5, 7, 7.5, 7.25]})

Adding a function for adding some Gaussian Noise:

In [ ]:
def gaussian_noise(x, mu=0, std=0.5):
    noise = np.random.normal(mu, std)
    x_noisy = x + noise
    return x_noisy 

Adding to df_initial the predicted values of the target:

In [ ]:
X_initial = df_initial[['Feature1', 'Feature2']]
X_transformed = poly_features.fit_transform(X_initial)

# get the predition from the Ground Truth model
df_initial['Target'] = gaussian_noise(reg.predict(X_transformed)[0])

## 4. Small algorithm to implement the Simulation

Source: https://github.com/experimental-design/bofire/blob/main/tutorials/basic_examples/Reaction_Optimization_Example.ipynb

This is the full implementation of the simulation. 

In [ ]:
max_experiments = 10
i = 0
done = False

while not done:
    i += 1 
    # create one single candidate based on the GP model by maximizing the acquisition function: 
    new_candidates = sobo_strategy.ask(1)

    # extract the predictors of the best candidate: 
    X_new = new_candidates[['Feature1', 'Feature2']]

    # transformation for the polynomial prediction:
    X_new_poly = poly_features.fit_transform(X_new)

    # obtaining the prediction on the new candidate and adding gaussian noise:
    y_true = reg.predict(X_new_poly)[0] 
    y_noise = gaussian_noise(y_true)
    
    # creating one row DataFrame:
    new_experiment_simulated = pd.DataFrame({'Feature1': X_new['Feature1'], 'Feature2': X_new['Feature2'], 'Target': y_noise})
    print(f"Iteration: {i}")
    print(new_experiment_simulated)

    # inserting in our Sobol strategy
    sobo_strategy.tell(new_experiment_simulated)

    if i > max_experiments:
        done = True

The resulting DataFrame contains all the iterations with the simulated results of the experiments.

In [ ]:
result = sobo_strategy.experiments
result

## 5. Graphical inspections of the results:

Understand at each iteration which is the result overall:

In [ ]:
result['Target'].plot()
plt.title('Bayesian Optimization Iterations')
plt.xlabel('Iterations')
plt.ylabel('Target')
# plt.savefig("iterations.png", dpi=300) ## use for saving figure
plt.show()

Function to plot in 3D the results:

In [ ]:
# Create the 3D scatterplot
def plotter_3D(result, name):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(result['Feature1'], result['Feature2'], result['Target'])

    # Set labels and title
    ax.zaxis.labelpad=-1
    ax.set_xlabel('$x_1$ = Feature1')
    ax.set_ylabel('$x_2$ = Feature2')
    ax.set_zlabel('$y$ = Target')
    ax.set_title('3D Bayesian Optimization')
    
    plt.figure()
    # fig.savefig(name, dpi=300)  ## use for saving figure

    # Show the plot
    plt.show()

In [ ]:
plotter_3D(result)

Function to create the counterplot of the results:

In [ ]:
def plot_countour(result, name):
    pH = np.round(np.linspace(6.5, 9, 101),2)
    Temperature = np.round(np.linspace(30, 55, 101),2)
    X1, X2 = np.meshgrid(Temperature, pH)
    X1_flat = X1.flatten()
    X2_flat = X2.flatten()
    X = pd.DataFrame(np.column_stack((X1_flat, X2_flat)), columns = ['Feature1', 'Feature2'])
    X_poly = poly_features.transform(X)
    Z_pred = reg.predict(X_poly)
    Z_pred = Z_pred.reshape(X1.shape)

    plt.figure(figsize=(7,5))
    fig = plt.contourf(X1, X2, Z_pred, levels=20, cmap='viridis')
    plt.scatter(result['Feature1'], result['Feature2'], color='black')
    plt.xlabel('$x_1$ = Feature1')
    plt.ylabel('$X_2$ = Feature2')
    plt.title('Bayesian Optimization Simulation')
    plt.colorbar(fig)
     # plt.savefig(name, dpi=300)
    plt.show()

In [ ]:
plot_countour(result)